In [1]:
import sys
import os
from scipy import linalg
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
sys.path.append("..")
# Enable interactive plotting in Jupyter Notebook

from utils.camera_utils import *

In [2]:
run_name = "RUN3"

spiegel_calib = "spiegel.mp4"
front_calib = "front.mp4"
stereo_calib = "stereo.mp4"

In [3]:
calibration_dir = f"../videos/{run_name}/calibration"
detection_dir = f"../videos/{run_name}/detection"
detection_files = os.listdir(detection_dir)
sC = stereoCamera(name=run_name, camera_size={0: (480, 240), 1: (480, 240)})
vL = videoLoader()


# Setting the Camera Positions

In [4]:
vL.load_video(os.path.join(detection_dir, detection_files[-1]))
frame = vL[10]
# 0 = mirror-cam 1 = frontal-cam
sC.set_anchor_point(frame, 0)
sC.set_anchor_point(frame, 1)
for frame in vL[:100]:
    frame = sC.draw_camera_region(frame)
    cv2.imshow("Frame", frame)
    frame0, frame1 = sC(frame)
    cv2.waitKey(10)
cv2.destroyAllWindows()


Left mouse button pressed! (609, 73)
Left mouse button pressed! (642, 517)


# Mirror Calibration

In [5]:
vL.load_video(os.path.join(calibration_dir, spiegel_calib))
frames = vL[100:110]
sC.calibrate(frames,0, rows=6, columns=8)

Left mouse button pressed! [(838, 106)]
Left mouse button pressed! [(838, 106), (1330, 524)]
rmse: 0.226910314291202
camera matrix:
 [[1.43293236e+03 0.00000000e+00 2.49964833e+02]
 [0.00000000e+00 1.49268482e+03 1.71374729e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
optimized camera matrix:
 [[1.43293236e+03 0.00000000e+00 2.49964833e+02]
 [0.00000000e+00 1.32447657e+03 1.65841419e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
distortion coeffs:
 [[ 3.87610834e+00 -2.37551233e+02  8.58792107e-02 -2.73148837e-02
  -4.98355192e+00]]


# Frontal Calibration

In [8]:
vL.load_video(os.path.join(calibration_dir, front_calib))
frames = vL[110:130]
sC.calibrate(frames,1, rows=6, columns=8)

Left mouse button pressed! [(589, 94)]
Left mouse button pressed! [(589, 94), (1316, 636)]
rmse: 0.3820196731592168
camera matrix:
 [[4.61146808e+03 0.00000000e+00 9.68909018e+02]
 [0.00000000e+00 4.51653969e+03 4.81350883e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
optimized camera matrix:
 [[4.51465805e+03 0.00000000e+00 9.53230397e+02]
 [0.00000000e+00 4.41853285e+03 4.66539332e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
distortion coeffs:
 [[ 2.13205802e-01  9.56544936e-01 -3.84874018e-02 -2.15231608e-02
  -1.46704375e+02]]


# Stereo Calibration

In [9]:
vL.load_video(os.path.join(calibration_dir, stereo_calib))
frames = vL[100::20]

In [10]:
frames = [frames[0]]

for i in range(2, 8, 1):
    print(os.path.join(calibration_dir, f"stereo{i}.mp4"))
    vL.load_video(os.path.join(calibration_dir, f"stereo{i}.mp4"))
    frames.append(vL[100])


../videos/RUN3/calibration/stereo2.mp4
../videos/RUN3/calibration/stereo3.mp4
../videos/RUN3/calibration/stereo4.mp4
../videos/RUN3/calibration/stereo5.mp4
../videos/RUN3/calibration/stereo6.mp4
../videos/RUN3/calibration/stereo7.mp4


In [11]:
frames = frames
#sC.load_from_yaml(f"{run_name}.yaml")
objectpoints, imgpoints_1, imgpoints_2, images = sC.stereo_calibrate(frames[:], rows=8, columns=10)

Current line: [593, 209]
Current line: [593, 209, 573, 392]
Current line: [613, 393]
Current line: [613, 393, 631, 209]
Current line: [669, 211]
Current line: [669, 211, 655, 394]
Current line: [697, 394]
Current line: [697, 394, 709, 211]
Current line: [747, 213]
Current line: [747, 213, 737, 394]
Current line: [778, 395]
Current line: [778, 395, 785, 212]
Current line: [823, 214]
Current line: [823, 214, 819, 395]
Current line: [861, 396]
Current line: [861, 396, 862, 211]
Current line: [901, 211]
Current line: [901, 211, 901, 394]
Current line: [942, 368]
Current line: [942, 368, 532, 365]
Current line: [538, 340]
Current line: [538, 340, 938, 345]
Current line: [937, 322]
Current line: [937, 322, 539, 317]
Current line: [543, 294]
Current line: [543, 294, 936, 300]
Current line: [937, 276]
Current line: [937, 276, 547, 270]
Current line: [548, 250]
Current line: [548, 250, 936, 254]
Current line: [935, 232]
Current line: [935, 232, 548, 226]
Current line: [548, 153]
Current line: [

In [ ]:
sC.conf["camera_matrix"][1]

In [ ]:
sC.conf

# Save Calibration to YAML

In [12]:
sC.save_to_yaml(f"{run_name}.yaml")

sC.save_to_yaml(f"{run_name}_exp.yaml")

In [ ]:
sC.load_from_yaml(f"{run_name}.yaml")

In [ ]:
sC.conf["anchor_point"], sC.name